# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

**Описание данных**  
Данные геологоразведки трёх регионов находятся в файлах, в данных есть следующие переменные:
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

**Условия задачи**
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**Загрузка необходимых функций и библиотек**

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Загрузка и подготовка данных

In [2]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Проверим, есть ли в данных пропуски и дубликаты:

In [4]:
data_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [5]:
data_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [6]:
data_3.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [7]:
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())
print(data_3.duplicated().sum())

0
0
0


В данных нет явных дубликатов и пропусков, далее проверим правильные ли типы у имеющихся переменных.

In [8]:
print(data_1.info())
print(data_2.info())
print(data_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

Типы переменных корректны. Для корректного обучения в дальнейшем уберем столбец id из данных, так как он не несет содержательного смысла и не поможет предсказывать что-либо.

In [9]:
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)
data_3 = data_3.drop(['id'], axis=1)

## Обучение и проверка модели

Разобьем данные на обучающую и валидационную выборки в соотношении 75:25, затем разделим данные на признаки и целевую переменную, целевой переменной будет объём запасов.

In [10]:
train_1, valid_1 = train_test_split(data_1, test_size=0.25, random_state=12345)

features_train_1 = train_1.drop(['product'], axis=1)
target_train_1 = train_1['product']

features_valid_1 = valid_1.drop(['product'], axis=1)
target_valid_1 = valid_1['product']



train_2, valid_2 = train_test_split(data_2, test_size=0.25, random_state=12345)

features_train_2 = train_2.drop(['product'], axis=1)
target_train_2 = train_2['product']

features_valid_2 = valid_2.drop(['product'], axis=1)
target_valid_2 = valid_2['product']



train_3, valid_3 = train_test_split(data_3, test_size=0.25, random_state=12345)

features_train_3 = train_3.drop(['product'], axis=1)
target_train_3 = train_3['product']

features_valid_3 = valid_3.drop(['product'], axis=1)
target_valid_3 = valid_3['product']

Напишем функцию для обучения модели, для условий постравленной задачи подходит модель линейной регрессии.

In [11]:
def models(features, target):
    model = LinearRegression()
    model.fit(features, target)
    return model

In [12]:
model_1 = models(features_train_1, target_train_1)
model_2 = models(features_train_2, target_train_2)
model_3 = models(features_train_3, target_train_3)

Далее сделаем предсказания на валидационной выборке и посмотрим на значения среднего запаса предсказанного сырья и RMSE модели. 

In [13]:
predictions_valid_1 = pd.Series(data=model_1.predict(features_valid_1), index=features_valid_1.index)
predictions_valid_2 = pd.Series(data=model_2.predict(features_valid_2), index=features_valid_2.index)
predictions_valid_3 = pd.Series(data=model_3.predict(features_valid_3), index=features_valid_3.index)

print('Средний запас предсказанного сырья в первом регионе:', predictions_valid_1.mean())
print('Средний запас предсказанного сырья во втором регионе:', predictions_valid_2.mean())
print('Средний запас предсказанного сырья в третьем регионе:', predictions_valid_3.mean())

Средний запас предсказанного сырья в первом регионе: 92.59256778438035
Средний запас предсказанного сырья во втором регионе: 68.728546895446
Средний запас предсказанного сырья в третьем регионе: 94.96504596800489


In [14]:
print("RMSE первой модели =", mean_squared_error(target_valid_1, predictions_valid_1) ** 0.5)
print("RMSE второй модели =", mean_squared_error(target_valid_2, predictions_valid_2) ** 0.5)
print("RMSE третьей модели =", mean_squared_error(target_valid_3, predictions_valid_3) ** 0.5)

RMSE первой модели = 37.5794217150813
RMSE второй модели = 0.893099286775617
RMSE третьей модели = 40.02970873393434


Третий регион имеет наибольшее среднее значение предсказанного запаса сырья, однако ошибка модели здесь наибольшая. Наименьшую ошибку имеет вторая модель, но среднее предсказанное значение - наименьшее. 

## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.


In [15]:
budget = 10000000000
price = 450000
points = 200
n = 500
prob = 0.025

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины, для этого бюджет поделим на произведение дохода на количество скважин.

In [16]:
print('Объём сырья для безубыточной разработки новой скважины равен', budget/(price*points))

Объём сырья для безубыточной разработки новой скважины равен 111.11111111111111


Сравним полученный объём сырья со средним запасом в каждом регионе. 

In [17]:
print('Средний объем запасов скважины в первом регионе: ', data_1['product'].mean())
print('Средний объем запасов скважины во втором регионе: ', data_2['product'].mean())
print('Средний объем запасов скважины в третьем регионе: ', data_3['product'].mean())

Средний объем запасов скважины в первом регионе:  92.50000000000001
Средний объем запасов скважины во втором регионе:  68.82500000000002
Средний объем запасов скважины в третьем регионе:  95.00000000000004


Средний объем запасов скважины во всех регионах имеет меньшее значение, чем объём сырья для безубыточной разработки новой скважины (92.5, 68.8 и 95 против 111.1). Однако стоит заметить, что у первого и третьего регионов разница не такая большая как у второго (средний объем запасов практически в 1,5 раза меньше, чем нужно).

## Расчёт прибыли и рисков 

Напишем функцию, которая будет расчитывать прибыль. Для расчета прибыли мы будем брать скважины с максимальными значениями предсказаний и просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.

In [18]:
def revenue(prediction, target):
    prediction_best = prediction.sort_values(ascending=False)
    target_best = target[prediction_best.index][:points]
    return (price*target_best.sum()) - budget

Используем технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли, для этого напишем функцию. 

In [19]:
def bootstrap(prediction, target):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        subsample_target = target.sample(n=500, replace=True, random_state=state)
        subsample_prediction = prediction[subsample_target.index]
        values.append(revenue(subsample_prediction, subsample_target))
    values = pd.Series(values)
    return values 

Зафиксируем распределение прибыли для трех регионов:

In [20]:
revenue_1 = bootstrap(predictions_valid_1, target_valid_1)
revenue_2 = bootstrap(predictions_valid_2, target_valid_2)
revenue_3 = bootstrap(predictions_valid_3, target_valid_3)

Найдем среднюю прибыль, а также 95%-й доверительный интервал и риск убытков.

In [21]:
print('Распределение прибыли для первого региона')
print('Средняя прибыль:', revenue_1.mean())
print('95%-й Доверительный интервал:', revenue_1.quantile(0.025), '-', revenue_1.quantile(0.975))
print('Риск убытков:', st.percentileofscore(revenue_1, 0), '%')

Распределение прибыли для первого региона
Средняя прибыль: 425938526.91059244
95%-й Доверительный интервал: -102090094.83793654 - 947976353.3583689
Риск убытков: 6.0 %


In [22]:
print('Распределение прибыли для второго региона')
print('Средняя прибыль:', revenue_2.mean())
print('95%-й Доверительный интервал:', revenue_2.quantile(0.025), '-', revenue_2.quantile(0.975))
print('Риск убытков:', st.percentileofscore(revenue_2, 0), '%')

Распределение прибыли для второго региона
Средняя прибыль: 515222773.4432899
95%-й Доверительный интервал: 68873225.37050176 - 931547591.2570494
Риск убытков: 1.0 %


In [23]:
print('Распределение прибыли для третьего региона')
print('Средняя прибыль:', revenue_3.mean())
print('95%-й Доверительный интервал:', revenue_3.quantile(0.025), '-', revenue_3.quantile(0.975))
print('Риск убытков:', st.percentileofscore(revenue_3, 0), '%')

Распределение прибыли для третьего региона
Средняя прибыль: 435008362.7827556
95%-й Доверительный интервал: -128880547.32978901 - 969706954.1802661
Риск убытков: 6.4 %


**Вывод**  
Исходя из проведенного анализа, выяснилось, что наибольшим риском убыточности обладает третий регион (показатель составил 6,4%), на втором месте по риску убытков расположился первый регион (показатель равень 6%). Меньше всего риск убыточности у второго региона, у него же наибольшая средняя прибыль, а если рассмотреть 95%-й доверительный интервал, то можно заметить, что прибыль в нем принимает положительные значения, следовательно, рекомендуется выбрать именно этот регион для разработки месторождения.  